In [1]:
from mvnc import mvncapi as mvnc
import cv2
import numpy as np
import time
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import yolo_ncs,ncs

# Load the base overlay
base = BaseOverlay("base.bit")

In [2]:
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure the HDMI output to the same resolution as the HDMI input
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

# Start the HDMI interfaces
hdmi_in.start()
hdmi_out.start()

In [3]:
# Open the Movidius NCS device
ncsdev = ncs.MovidiusNCS()

# Load the graph file
if ncsdev.load_graph('../models/graph'):
    print('Graph file loaded to Movidius NCS')

Graph file loaded to Movidius NCS


In [4]:
# The input frame width and height
frame_in_width = hdmi_in.mode.width
frame_in_height = hdmi_in.mode.height

# The frame width and height required by the YOLO network
yolo_width = 448
yolo_height = 448

# The subframe indicies to cut-out for the YOLO network (the middle of the input frame)
frame_top = (frame_in_height-yolo_height)//2
frame_bot = frame_in_height-frame_top
frame_left = (frame_in_width-yolo_width)//2
frame_right = frame_in_width-frame_left

In [ ]:
while True:
    # Get a frame from HDMI input
    frame = hdmi_in.readframe()
    
    # Cut out a sub-frame for NCS (448x448) and divide all RGB values by 255
    small_frame = frame[frame_top:frame_bot,frame_left:frame_right,:]
    ncs_frame = small_frame.copy()/255.0

    # Send the frame to the NCS
    ncsdev.graph.LoadTensor(ncs_frame.astype(np.float16), 'user object')
    out, userobj = ncsdev.graph.GetResult()
    
    # Interpret results and draw the boxes on the image
    results = yolo_ncs.interpret_output(out.astype(np.float32), small_frame.shape[1], small_frame.shape[0]) # fc27 instead of fc12 for yolo_small
    img_res = yolo_ncs.draw_boxes(small_frame, results, small_frame.shape[1], small_frame.shape[0])

    # Copy labelled image into output frame
    frame_out = hdmi_out.newframe()
    frame_out[frame_top:frame_bot,frame_left:frame_right,:] = img_res[:,:,:]
    hdmi_out.writeframe(frame_out)